In [1]:
import sys
import os

print("현재 작업 디렉토리:", os.getcwd())
project_root = os.path.abspath("../../../")
print("프로젝트 루트 디렉토리:", project_root)

if project_root not in sys.path:
    sys.path.append(project_root)

현재 작업 디렉토리: c:\mygit\SkyRoot\src\analysis\route
프로젝트 루트 디렉토리: c:\mygit\SkyRoot


In [ ]:
import requests
import polyline

# 1. Google Maps Client
from src.config.settings import API_KEY
url = "https://maps.googleapis.com/maps/api/directions/json"
params = {
    "origin": "37.53346389,127.0718772",  # 서울 시
    "destination": "37.4842721,126.4240467",  # 남산타워
    "departure_time": "now",
    "mode": "transit",
    "transit_mode": "bus",
    "transit_routing_preference": "fewer_transfers",
    "alternatives": "true",
    "language": "ko",
    "key": API_KEY
}

res = requests.get(url, params=params)
data = res.json()

In [ ]:
import pandas as pd

trip_no = "TRIP_430377"

rows = []
for route_no, route in enumerate(data["routes"], start=1):
    for step in route["legs"][0]["steps"]:
        row = {
            "TRIP_NO": trip_no,
            "ROUTE_NO": route_no,
            "DISTANCE": step["distance"]["value"],     # meters
            "DURATION": step["duration"]["value"],     # seconds
            "START_LNG": step["start_location"]["lng"],
            "START_LAT": step["start_location"]["lat"],
            "END_LNG": step["end_location"]["lng"],
            "END_LAT": step["end_location"]["lat"],
            "POLYLINE": step["polyline"]["points"],
            "TRAVEL_MODE": step["travel_mode"],
            "BUS_NAME": None,
            "BUS_TYPE": None
        }

        # 🚍 대중교통(버스)일 경우
        if step["travel_mode"] == "TRANSIT":
            transit = step.get("transit_details", {})
            line = transit.get("line", {})
            row["BUS_NAME"] = line.get("short_name")
            row["BUS_TYPE"] = line.get("name")

        rows.append(row)

travel_total_df = pd.DataFrame(rows)


In [39]:
trip_no = "TRIP_430377"

rows = []

for route_no, route in enumerate(data["routes"], start=1):
    for step in route["legs"][0]["steps"]:
        travel_mode = step["travel_mode"]
        bus_name = None
        bus_type = None

        if travel_mode == "TRANSIT":
            line = step["transit_details"]["line"]
            bus_name = line.get("short_name")
            bus_type = line.get("name")

        decoded_points = polyline.decode(step["polyline"]["points"])

        for point_seq, (lat, lng) in enumerate(decoded_points, start=1):
            rows.append({
                "TRIP_NO": trip_no,
                "STEP_NO": route_no,
                "POINT_SEQ": point_seq,
                "LAT": lat,
                "LNG": lng,
                "TRAVEL_MODE": travel_mode,
                "BUS_NAME": bus_name,
                "BUS_TYPE": bus_type
            })

travel_point_df = pd.DataFrame(rows)
